Experiment. Doel: uit huidige republic data set de grootst mogelijke doorlopende 'talige' tekst extraheren, en alle fysische en logische structuur opslaan als standoff annotatie tov daarvan. Dit met behoud van een 'browsable' structuur ('manifest') en links met image regions op line niveau. En als het even kan een relatie met de page XMLs op het niveau van XML elementen (+ selectors).

Stap 1: lees 1705-01-02 session json in in een Python datastructuur.

In [1]:
import json

# read file
with open('../data/sessions/meeting-1705-01-02-session-1.json', 'r') as myfile:
    session_file=myfile.read()

# parse file
session_data = json.loads(session_file)

In [2]:
session_data.keys()

dict_keys(['_index', '_type', '_id', '_version', '_seq_no', '_primary_term', 'found', '_source'])

In [3]:
column_data = session_data['_source']['columns']
type(column_data)

# column_data is a list of dicts

list

In [4]:
source_data = session_data['_source']
type(source_data)

dict

Stap 2: doorloop die datastructuur 'depth-first', en 'post-order' (visit node after it's children)

In [5]:
# node has children in the form of a list (we need list, because this represents order of elements)
# marijn's json also contains meta info for the node, next to its children; therefore dicts are used for children

# node: a dict containing a list of children, different keys at each level are used for these children.
# node_label: a string that identifies the level of the node in the tree (e.g. lines, textregions, columns, sessions)
# These keys represent 'physical' structure elements: columns, textregions, lines

# NOTE: quick and dirty implementation, just assume first list in dict contains the children

def traverse(node,node_label):
    # find the list that represents the children, each child is a dict, assume first list is the correct one
    children = []
    label_of_children = ''
    for key,val in node.items():
        if (type(val) == list):
            children = val
            label_of_children = key
            break      
    
    if len(children) == 0:        # if no children, do your 'leaf node thing'
        if node['text'] is None:
            print('')
        else:
            print(node['text'])
    else:                         # if non-leaf node, first visit children
        for child in children:
            traverse(child,label_of_children)
    print(node_label)
    return

In [6]:
traverse(source_data,'sessions')

Jovis den 1. Januari 1705.
lines
Nihil actum est.
lines

lines
7 61
lines
Veneris den 2. Januarii 1705.
lines
1951
lines
P R ES I D E,
lines
vooila
lines
textregions
7 In:
lines
5
lines
textregions
De Heer van Gent.
lines
P R AE S E.N T I B U S,
lines
De Heeren Ham.
lines
V. anden Honert ‚ Heemskerck, Meerens , Raedt-
lines
Pensionaris Heynsius,
lines
Becker , met een extraordinaris Gedeputeerde uyt
lines
de Provincie van Zeelandt.
lines
Wellandt, Renswoude , Velthuysen, met een ex-
lines
traordinaris Gedeputeerde uy de Provincie van
lines
Utrecht.
lines
Dau Tour.
lines
Wichers, Gockinga.
lines
textregions
ai
lines
a
lines
Rinm gE Resolutien eergisteren genomen
lines
textregions
jp En geleen en geresumeert , ge-
lines
|
lines
A He steert zyn de Depesches daer uyt re-
lines
a ) sulterende.
lines
Ontfangen een Missive van den Resident Bil-
lines
derbeeck, geschreven tot Keulen den dertigsten
lines
der voorlede maendt ‚ houdende advertentie :
lines
Waer op geen resolutie is gevallen.
line

lines
mogen werden in het werck gesteldt om den ge.
lines
amtlden Peer € de 5 alvé te verh 0 sen. Waer
lines

lines
i ii van het voorsch re ve geproponeerde k e nnisse
lines
sal werden gegeven aen den Heere van Vryber.
lines
gen, haer Hoogh Mogende extraordinaris En.
lines

lines
ginne van Groot Brittannien, om daer het van
lines
dienst al konnen wesen devoiren aen te wenden,
lines
ten eynde den gemelden Pierre de Satvé tegen een
lines
van gemelde Roomsche Priesters moge werden
lines
uytgewisselt ; en dat dien onvermindert Copy.
lines
vais het 190 esch ir eve 8 eproponeer de gestelde sal
lines
werden in handen van de Heeren Hain, en an.
lines
dere haer Hoogh Mog ende Gedeputeerden tot de
lines
Misicaire saken, om te visiteren, examineren
lines
eiide van alles alhier ter Vergaderinge rapport te
lines
doen.
lines
Is gehoort het rapport van de Heeren Ham,
lines
ende andere haer Hoogh Mogende Gedeputeer.
lines
den tot de Misitaire saecken hebbende, in ge.
lines
volge en tot voldoeninge van

Waer op gedelibereert zynde is oöetgevonden en
lines
verftaen, dat Copye van de vóor{chreve Memo-
lines
textregions
columns
rie oesteldt sal werden in handen van de Heeren
lines
van Gent, en andere haer Hoogh Mogende Ge-
lines
deputeerden tot de Buytenlandtsche saken, om
lines
over het voorschreve subject te spreken met den
lines
Heere Baron van Schmetteau , Plenipotentiaris
lines
van sijne Majesteyt den Koningh van Pruyssen,
lines
ende van alles alhier ter Vergaderinge rapport te
lines
doen.
lines
Ontfangen een Missive van de Heeren Gede-
lines
puteerde Staten des Nymeeghschen Quartiers
lines
geschreven tot Nymegen den een en dertighsten
lines
der voorlede maendt, houdende, der selver re-
lines
scriptie op haer Hoogh Mogende Missive van den
lines
negen en twintiehsten daer te vooren, daer by
lines
haer Hoogh Mogende de gemelde Heeren Gede-
lines
puteerde Staten hadden versocht die voorsienin-
lines
ge te willen doen, dat de ses Compagnien van
lines
het Regiment van Schratenbach , in d

STAVAZA: DFS traversal van marijn's json lijkt nu te lukken, en op het eerste gezicht de complete tekst per line in de juiste volgorde op te leveren. In het vervolg ga ik nu stap voor stap alle extra informatie per line in een tabel toevoegen.

Stap 3: verzamel alle line teksten in de juiste volgorde in een list. Voorlopige 'selector' voor die tekst is dan de list index.

In [7]:
# version 2 of traverse function, replace print statements with collection of results in a list

def traverse2(node,node_label,text):
    # find the list that represents the children, each child is a dict, assume first list is the correct one
    children = []
    label_of_children = ''
    for key,val in node.items():
        if (type(val) == list):
            children = val
            label_of_children = key
            break      
    
    if len(children) == 0:        # if no children, do your 'leaf node thing'
        if node['text'] is None:
            text.append('\n')
        else:
            text.append(node['text'])
    else:                         # if non-leaf node, first visit children
        for child in children:
            traverse2(child,label_of_children,text)
    print(node_label)
    return

In [8]:
text_array = []
traverse2(source_data,'sessions',text_array)

lines
lines
lines
lines
lines
lines
lines
lines
textregions
lines
lines
textregions
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
textregions
lines
lines
lines
textregions
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
textregions
columns
lines
lines
lines
textregions
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
textregions
columns
lines
lines
lines
lines
lines
textregions
lines
lines
lines
lines
lines
lines
lines
lines
lines
textregions
columns
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
lines
text

In [9]:
text_array

['Jovis den 1. Januari 1705.',
 'Nihil actum est.',
 '\n',
 '7 61',
 'Veneris den 2. Januarii 1705.',
 '1951',
 'P R ES I D E,',
 'vooila',
 '7 In:',
 '5',
 'De Heer van Gent.',
 'P R AE S E.N T I B U S,',
 'De Heeren Ham.',
 'V. anden Honert ‚ Heemskerck, Meerens , Raedt-',
 'Pensionaris Heynsius,',
 'Becker , met een extraordinaris Gedeputeerde uyt',
 'de Provincie van Zeelandt.',
 'Wellandt, Renswoude , Velthuysen, met een ex-',
 'traordinaris Gedeputeerde uy de Provincie van',
 'Utrecht.',
 'Dau Tour.',
 'Wichers, Gockinga.',
 'ai',
 'a',
 'Rinm gE Resolutien eergisteren genomen',
 'jp En geleen en geresumeert , ge-',
 '|',
 'A He steert zyn de Depesches daer uyt re-',
 'a ) sulterende.',
 'Ontfangen een Missive van den Resident Bil-',
 'derbeeck, geschreven tot Keulen den dertigsten',
 'der voorlede maendt ‚ houdende advertentie :',
 'Waer op geen resolutie is gevallen.',
 'Ontfangen een Missive van haer Hoogh Mo-',
 'gende Gecommitteerde Hulft i geschreven',
 '\n',
 '120 D noen A

Stap 4: per line, maak een annotion_info dict aan, sla daarin id, label en image_coords op.

In [10]:
# version 3 of traverse function, create annotation_info dicts per text line, and add id and label for line level

def traverse3(node,node_label,text):
    # find the list that represents the children, each child is a dict, assume first list is the correct one
    children = []
    label_of_children = ''
    for key,val in node.items():
        if (type(val) == list):
            children = val
            label_of_children = key
            break      
    
    if len(children) == 0:        # if no children, do your 'leaf node thing'
        annotation_info = {'label' : node_label,'id' : node['id'],'image_coords' : node['coords']}
        node_text = node['text']
        
        if node_text is None:
            node_text = '\n'

        text.append((node_text,annotation_info))
    else:                         # if non-leaf node, first visit children
        for child in children:
            traverse3(child,label_of_children,text)
    return

In [11]:
text_array = []
traverse3(source_data,'sessions',text_array)
text_array[-7:]

[('sal werden aengeschreven , den Hertogh van',
  {'label': 'lines',
   'id': 'NL-HaNA_1.01.02_3760_0012-page-23-col-1-tr-0-line-46',
   'image_coords': {'left': 3539,
    'right': 4424,
    'top': 2601,
    'bottom': 2681,
    'height': 80,
    'width': 885}}),
 ('Marlborou gee versoecken om insgelijks Gen or -',
  {'label': 'lines',
   'id': 'NL-HaNA_1.01.02_3760_0012-page-23-col-1-tr-0-line-47',
   'image_coords': {'left': 3542,
    'right': 4419,
    'top': 2650,
    'bottom': 2726,
    'height': 76,
    'width': 877}}),
 ('dre te senden aen de Generacls van de Deensche',
  {'label': 'lines',
   'id': 'NL-HaNA_1.01.02_3760_0012-page-23-col-1-tr-0-line-48',
   'image_coords': {'left': 3539,
    'right': 4418,
    'top': 2697,
    'bottom': 2778,
    'height': 81,
    'width': 879}}),
 ('en Hessische Troupes d ten eynde de selve mogen',
  {'label': 'lines',
   'id': 'NL-HaNA_1.01.02_3760_0012-page-23-col-1-tr-0-line-49',
   'image_coords': {'left': 3537,
    'right': 4419,
    'top':

TODO: deal met textregion, column, ... lagen.

In [12]:
# version 4 of traverse function, create annotation_info dicts for each node, and add id, label and coords

def traverse4(node,node_label,text,annotations):
    # find the list that represents the children, each child is a dict, assume first list is the correct one
    children = []
    label_of_children = ''
    for key,val in node.items():
        if (type(val) == list):
            children = val
            label_of_children = key
            break 
    
    if 'coords' in node:
        coords = node['coords']
    else:
        coords = None
    
    annotation_info = {'label' : node_label,'image_coords': coords,'begin_index' : len(text)}
    if len(children) == 0:        # if no children, do your 'leaf node thing'
        annotation_info['id'] = node['id']
        annotation_info['end_index'] = len(text)
        node_text = node['text']
        
        if node_text is None:
            node_text = '\n'

        text.append(node_text)
    else:                         # if non-leaf node, first visit children     
        for child in children:
            traverse4(child,label_of_children,text,annotations)
        
        # ONDERSTAANDE IS SMERIG, hangt van onzekere aannames af
        for k in node['metadata'].keys():
            idkey = ''
            if k.endswith('id'):
                idkey = k
                break
        annotation_info['id'] = node['metadata'][idkey]
        annotation_info['end_index'] = len(text)-1    # after child text segments are added
        
    annotations.append(annotation_info)
    return

In [13]:
text_array = []
annotation_array = []
traverse4(source_data,'sessions',text_array,annotation_array)
annotation_array[-7:]

[{'label': 'lines',
  'image_coords': {'left': 3537,
   'right': 4419,
   'top': 2748,
   'bottom': 2826,
   'height': 78,
   'width': 882},
  'begin_index': 1053,
  'id': 'NL-HaNA_1.01.02_3760_0012-page-23-col-1-tr-0-line-49',
  'end_index': 1053},
 {'label': 'lines',
  'image_coords': {'left': 3533,
   'right': 4421,
   'top': 2796,
   'bottom': 2876,
   'height': 80,
   'width': 888},
  'begin_index': 1054,
  'id': 'NL-HaNA_1.01.02_3760_0012-page-23-col-1-tr-0-line-50',
  'end_index': 1054},
 {'label': 'lines',
  'image_coords': {'left': 3532,
   'right': 3693,
   'top': 2847,
   'bottom': 2902,
   'height': 55,
   'width': 161},
  'begin_index': 1055,
  'id': 'NL-HaNA_1.01.02_3760_0012-page-23-col-1-tr-0-line-51',
  'end_index': 1055},
 {'label': 'lines',
  'image_coords': {'left': 3772,
   'right': 3814,
   'top': 2871,
   'bottom': 2924,
   'height': 53,
   'width': 42},
  'begin_index': 1056,
  'id': 'NL-HaNA_1.01.02_3760_0012-page-23-col-1-tr-0-line-52',
  'end_index': 1056},
 

In [14]:
text_array[-7:]

['sal werden aengeschreven , den Hertogh van',
 'Marlborou gee versoecken om insgelijks Gen or -',
 'dre te senden aen de Generacls van de Deensche',
 'en Hessische Troupes d ten eynde de selve mogen',
 'obedieren aen de ordres van gemelden Grave aen',
 'Noyelle.',
 '9']

In [15]:
print(len(text_array))
print(len(annotation_array))

1057
1106


Nog te doen: iiif_url meenemen op de 'juiste' wijze (kan ik ook iets met pageXML reference?)
Daarna code wat netter maken
Daarna concateneren van alle text lines tot 1 tekst aanpakken
Daarna: evalueer op genericiteit, annoteerbaarheid, navigeerbare structuur aanwezig?

In [16]:
# version 5 of traverse function, now add annotation_info for label 'scanpage'

def traverse5(node,node_label,text,annotations):
    # find the list that represents the children, each child is a dict, assume first list is the correct one
    children = []
    label_of_children = ''
    for key,val in node.items():
        if (type(val) == list):
            children = val
            label_of_children = key
            break 
    
    if 'coords' in node:
        coords = node['coords']
    else:
        coords = None
    
    begin_index = len(text)
    annotation_info = {'label' : node_label,'image_coords': coords,'begin_index' : begin_index}
    if len(children) == 0:        # if no children, do your 'leaf node thing'
        annotation_info['id'] = node['id']
        annotation_info['end_index'] = len(text)
        node_text = node['text']
        
        if node_text is None:
            node_text = '\n'

        text.append(node_text)
    else:                         # if non-leaf node, first visit children     
        for child in children:
            traverse5(child,label_of_children,text,annotations)
        
        # ONDERSTAANDE IS SMERIG, hangt van onzekere aannames af
        for k in node['metadata'].keys():
            idkey = ''
            if k.endswith('id'):
                idkey = k
                break
        annotation_info['id'] = node['metadata'][idkey]
        
        end_index = len(text)-1
        annotation_info['end_index'] = end_index    # after child text segments are added
        
        # if node contains iiif_url, create extra annotation_info for 'scanpage'
        if 'iiif_url' in node['metadata']:
            scan_annot_info = {'label':'scanpage','iiif_url':node['metadata']['iiif_url'],'begin_index':begin_index,'end_index':end_index}
            scan_annot_info['scan_num'] = node['metadata']['scan_num']
            annotations.append(scan_annot_info)
        
    annotations.append(annotation_info)
    return

In [17]:
text_array = []
annotation_array = []
traverse5(source_data,'sessions',text_array,annotation_array)
annotation_array[-7:]

[{'label': 'lines',
  'image_coords': {'left': 3533,
   'right': 4421,
   'top': 2796,
   'bottom': 2876,
   'height': 80,
   'width': 888},
  'begin_index': 1054,
  'id': 'NL-HaNA_1.01.02_3760_0012-page-23-col-1-tr-0-line-50',
  'end_index': 1054},
 {'label': 'lines',
  'image_coords': {'left': 3532,
   'right': 3693,
   'top': 2847,
   'bottom': 2902,
   'height': 55,
   'width': 161},
  'begin_index': 1055,
  'id': 'NL-HaNA_1.01.02_3760_0012-page-23-col-1-tr-0-line-51',
  'end_index': 1055},
 {'label': 'lines',
  'image_coords': {'left': 3772,
   'right': 3814,
   'top': 2871,
   'bottom': 2924,
   'height': 53,
   'width': 42},
  'begin_index': 1056,
  'id': 'NL-HaNA_1.01.02_3760_0012-page-23-col-1-tr-0-line-52',
  'end_index': 1056},
 {'label': 'textregions',
  'image_coords': {'left': 3532,
   'right': 4478,
   'top': 366,
   'bottom': 2924,
   'height': 2558,
   'width': 946},
  'begin_index': 1004,
  'id': 'NL-HaNA_1.01.02_3760_0012-page-23-col-1-tr-0',
  'end_index': 1056},
 {

Los idee: heeft het zin om een 'node' class te definieren, die verschillende implementaties kent?

In [18]:
# use a generator to create a list of only scanpage annotation_info dicts
    
scan_page_annots = [ann_info for ann_info in annotation_array if ann_info['label'] == 'scanpage']
scan_page_annots
    

[{'label': 'scanpage',
  'iiif_url': 'https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0008.jpg/2402,1750,1094,1616/full/0/default.jpg',
  'begin_index': 0,
  'end_index': 34,
  'scan_num': 8},
 {'label': 'scanpage',
  'iiif_url': 'https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0008.jpg/3353,516,1109,2856/full/0/default.jpg',
  'begin_index': 35,
  'end_index': 68,
  'scan_num': 8},
 {'label': 'scanpage',
  'iiif_url': 'https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0008.jpg/3865,726,616,1116/full/0/default.jpg',
  'begin_index': 69,
  'end_index': 82,
  'scan_num': 8},
 {'label': 'scanpage',
  'iiif_url': 'https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0009.jpg/299,259,1118,3114/full/0/default.jpg',
  'begin_index': 83,
  'end_index': 143,
  'scan_num': 9},
 {'label': 'scanpage',
  'iiif_url': 'https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0009.jpg

In [19]:
# use groupBy on a list of dicts (zie Python cookbook 1.15)
from operator import itemgetter
from itertools import groupby

# first sort on scan_num
scan_page_annots.sort(key=itemgetter('scan_num'))

# iterate in groups
aggregated_scan_annots = []

for scan_num, items in groupby(scan_page_annots, key=itemgetter('scan_num')):
    # first, convert the 'items' iterator to a list, to able to use it twice (iterators can be used once)
    itemlist = list(items)
    
    # copy the item with the lowest begin_index
    aggr_scan_annot = min(itemlist, key=itemgetter('begin_index')).copy()
    
    # replace 'end_index' with the highest end_index in the group
    max_end_index = max(itemlist, key=itemgetter('end_index'))['end_index']
    aggr_scan_annot['end_index'] = max_end_index
    
    # iiif_urls in the group refer to different column regions of the same scan, strip coord part of url
    
    # add to result
    aggregated_scan_annots.append(aggr_scan_annot)

aggregated_scan_annots


[{'label': 'scanpage',
  'iiif_url': 'https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0008.jpg/2402,1750,1094,1616/full/0/default.jpg',
  'begin_index': 0,
  'end_index': 82,
  'scan_num': 8},
 {'label': 'scanpage',
  'iiif_url': 'https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0009.jpg/299,259,1118,3114/full/0/default.jpg',
  'begin_index': 83,
  'end_index': 324,
  'scan_num': 9},
 {'label': 'scanpage',
  'iiif_url': 'https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0010.jpg/280,274,1107,3116/full/0/default.jpg',
  'begin_index': 325,
  'end_index': 572,
  'scan_num': 10},
 {'label': 'scanpage',
  'iiif_url': 'https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0011.jpg/284,266,1111,3108/full/0/default.jpg',
  'begin_index': 573,
  'end_index': 822,
  'scan_num': 11},
 {'label': 'scanpage',
  'iiif_url': 'https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_001

Next step: adjust and complete urls of image and pageXML.
Then: subtract scan_page_annots from, add aggreted_scan_annots to annotation_array

In [20]:
import re

for scan_ann in aggregated_scan_annots:
    scan_ann['iiif_url'] = re.sub(r'(\d+),(\d+),(\d+),(\d+)/(full)', r'\5/,\4', scan_ann['iiif_url'])

In [21]:
aggregated_scan_annots

[{'label': 'scanpage',
  'iiif_url': 'https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0008.jpg/full/,1616/0/default.jpg',
  'begin_index': 0,
  'end_index': 82,
  'scan_num': 8},
 {'label': 'scanpage',
  'iiif_url': 'https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0009.jpg/full/,3114/0/default.jpg',
  'begin_index': 83,
  'end_index': 324,
  'scan_num': 9},
 {'label': 'scanpage',
  'iiif_url': 'https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0010.jpg/full/,3116/0/default.jpg',
  'begin_index': 325,
  'end_index': 572,
  'scan_num': 10},
 {'label': 'scanpage',
  'iiif_url': 'https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0011.jpg/full/,3108/0/default.jpg',
  'begin_index': 573,
  'end_index': 822,
  'scan_num': 11},
 {'label': 'scanpage',
  'iiif_url': 'https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0012.jpg/full/,3114/0/default.jpg',
  'begin_index': 

Verwijder nog scan_page_annots en voeg aggregated_scan_annots toe aan annotation_array

In [22]:
len(annotation_array)

1125

In [23]:
len(scan_page_annots)

19

In [24]:
len(aggregated_scan_annots)

5

In [25]:
annotation_array = [ann for ann in annotation_array if ann not in scan_page_annots]

In [26]:
len(annotation_array)

1106

In [27]:
annotation_array.extend(aggregated_scan_annots)

In [28]:
len(annotation_array)

1111

Observatie: 'page_versions' in Marijn's json bevat specifieke identifiers en checksum van bijbehorende pageXML files (neem ik even aan), en daarnaast page_id's. Laat maar even een 'resolvable link' naar in een instantie van de PageXML zitten voor later, maar voeg wel nog een 'page' node toe, inclusief begin_index en end_index daarvoor.

In [29]:
page_data = session_data['_source']['page_versions']
page_data[:2]

[{'id': 'd8e422fd-634c-4fd5-8cc9-98fef00645a2',
  'fileId': '8b6c649d-f373-4f80-8aa8-97948506aa46',
  'createdAt': '2020-09-25T09:41:07',
  'contentsSha': '46c294afbf6374858f142bdc12f1795b7501b03aaff483ef5ca2a85f',
  'page_id': 'NL-HaNA_1.01.02_3760_0008-page-15'},
 {'id': '335ab830-b878-487d-a683-98d559171500',
  'fileId': '1d3cbaa0-cfdd-4ce0-8863-761a51cbebe2',
  'createdAt': '2020-09-25T09:40:53',
  'contentsSha': '2a4c7dffe43e1d4064357cac91b6bcc90c75e736eaeb86f781294f8d',
  'page_id': 'NL-HaNA_1.01.02_3760_0009-page-16'}]

In [30]:
[ pg['page_id'] for pg in page_data ] 

['NL-HaNA_1.01.02_3760_0008-page-15',
 'NL-HaNA_1.01.02_3760_0009-page-16',
 'NL-HaNA_1.01.02_3760_0009-page-17',
 'NL-HaNA_1.01.02_3760_0010-page-18',
 'NL-HaNA_1.01.02_3760_0010-page-19',
 'NL-HaNA_1.01.02_3760_0011-page-20',
 'NL-HaNA_1.01.02_3760_0011-page-21',
 'NL-HaNA_1.01.02_3760_0012-page-22',
 'NL-HaNA_1.01.02_3760_0012-page-23']

In [31]:
# generator
page_identifiers = (pg['page_id'] for pg in page_data)
page_annots = [{'label' : 'pages','id' : page_id} for page_id in page_identifiers]
page_annots

[{'label': 'pages', 'id': 'NL-HaNA_1.01.02_3760_0008-page-15'},
 {'label': 'pages', 'id': 'NL-HaNA_1.01.02_3760_0009-page-16'},
 {'label': 'pages', 'id': 'NL-HaNA_1.01.02_3760_0009-page-17'},
 {'label': 'pages', 'id': 'NL-HaNA_1.01.02_3760_0010-page-18'},
 {'label': 'pages', 'id': 'NL-HaNA_1.01.02_3760_0010-page-19'},
 {'label': 'pages', 'id': 'NL-HaNA_1.01.02_3760_0011-page-20'},
 {'label': 'pages', 'id': 'NL-HaNA_1.01.02_3760_0011-page-21'},
 {'label': 'pages', 'id': 'NL-HaNA_1.01.02_3760_0012-page-22'},
 {'label': 'pages', 'id': 'NL-HaNA_1.01.02_3760_0012-page-23'}]

Volgende stap: voeg begin_index en end_index toe, op basis van de kolomwaarden (lastig?). Eventueel, als ik die kolommen heb, kan ik ook een enclosing rectangle bepalen, voor de image_coords van de pagina.

Om die kolommen te bepalen: ik weet scan_num, voor de volgorde kan ik column begin/end gebruiken, voor de page break op de pagina heb ik geen goede info (?).

Misschien aanleiding voor een generieke functie: vind alle annotaties op positie n. 

In [32]:
def annotations_at_index(index,label=None):
    annots_at_index = [ann_info for ann_info in annotation_array if ann_info['begin_index'] <= index <= ann_info['end_index']]
    if label != None:
        annots_at_index = scan_page_annots = [ai for ai in annots_at_index if ai['label'] == label]
    
    return annots_at_index


In [33]:
annotations_at_index(800)

[{'label': 'lines',
  'image_coords': {'left': 3491,
   'right': 4369,
   'top': 2182,
   'bottom': 2251,
   'height': 69,
   'width': 878},
  'begin_index': 800,
  'id': 'NL-HaNA_1.01.02_3760_0011-page-21-col-1-tr-0-line-38',
  'end_index': 800},
 {'label': 'textregions',
  'image_coords': {'left': 3466,
   'right': 4403,
   'top': 347,
   'bottom': 3014,
   'height': 2667,
   'width': 937},
  'begin_index': 762,
  'id': 'NL-HaNA_1.01.02_3760_0011-page-21-col-1-tr-0',
  'end_index': 816},
 {'label': 'columns',
  'image_coords': {'left': 3462,
   'right': 4403,
   'top': 347,
   'bottom': 3259,
   'height': 2912,
   'width': 941},
  'begin_index': 762,
  'id': 'NL-HaNA_1.01.02_3760_0011-page-21-col-1',
  'end_index': 822},
 {'label': 'sessions',
  'image_coords': None,
  'begin_index': 0,
  'id': 'meeting-1705-01-02-session-1',
  'end_index': 1056},
 {'label': 'scanpage',
  'iiif_url': 'https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0011.jpg/full/,3108/0/def

In [34]:
page_id = 'NL-HaNA_1.01.02_3760_0010-page-19'
result = re.search(r'(\d+)-page-', page_id)
int(result.group(1))

10

In [35]:
scanpage_for_scan_num = [ai for ai in annotation_array if 'scan_num' in ai.keys() and ai['scan_num'] == 10]
scanpage_for_scan_num

[{'label': 'scanpage',
  'iiif_url': 'https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0010.jpg/full/,3116/0/default.jpg',
  'begin_index': 325,
  'end_index': 572,
  'scan_num': 10}]

for each page_annot, derive scan_num from page_id, retrieve corresponding scanpage annot and copy begin_index and end_index

In [36]:
for pa in page_annots:
    scan_num = int(re.search(r'(\d+)-page-', pa['id']).group(1))
    scanpage_for_scan_num = [ai for ai in annotation_array if 'scan_num' in ai.keys() and ai['scan_num'] == scan_num]
    pa['begin_index'] = scanpage_for_scan_num[0]['begin_index']
    pa['end_index'] = scanpage_for_scan_num[0]['end_index']
    pa['indexesByContainment'] = True
    
page_annots

[{'label': 'pages',
  'id': 'NL-HaNA_1.01.02_3760_0008-page-15',
  'begin_index': 0,
  'end_index': 82,
  'indexesByContainment': True},
 {'label': 'pages',
  'id': 'NL-HaNA_1.01.02_3760_0009-page-16',
  'begin_index': 83,
  'end_index': 324,
  'indexesByContainment': True},
 {'label': 'pages',
  'id': 'NL-HaNA_1.01.02_3760_0009-page-17',
  'begin_index': 83,
  'end_index': 324,
  'indexesByContainment': True},
 {'label': 'pages',
  'id': 'NL-HaNA_1.01.02_3760_0010-page-18',
  'begin_index': 325,
  'end_index': 572,
  'indexesByContainment': True},
 {'label': 'pages',
  'id': 'NL-HaNA_1.01.02_3760_0010-page-19',
  'begin_index': 325,
  'end_index': 572,
  'indexesByContainment': True},
 {'label': 'pages',
  'id': 'NL-HaNA_1.01.02_3760_0011-page-20',
  'begin_index': 573,
  'end_index': 822,
  'indexesByContainment': True},
 {'label': 'pages',
  'id': 'NL-HaNA_1.01.02_3760_0011-page-21',
  'begin_index': 573,
  'end_index': 822,
  'indexesByContainment': True},
 {'label': 'pages',
  'id

Add page annotations to annotation_array

In [37]:
print(len(annotation_array))
print(annotation_array.extend(page_annots))
print(len(annotation_array))

1111
None
1120


In [38]:
annotation_array[-20:]

[{'label': 'lines',
  'image_coords': {'left': 3533,
   'right': 4421,
   'top': 2796,
   'bottom': 2876,
   'height': 80,
   'width': 888},
  'begin_index': 1054,
  'id': 'NL-HaNA_1.01.02_3760_0012-page-23-col-1-tr-0-line-50',
  'end_index': 1054},
 {'label': 'lines',
  'image_coords': {'left': 3532,
   'right': 3693,
   'top': 2847,
   'bottom': 2902,
   'height': 55,
   'width': 161},
  'begin_index': 1055,
  'id': 'NL-HaNA_1.01.02_3760_0012-page-23-col-1-tr-0-line-51',
  'end_index': 1055},
 {'label': 'lines',
  'image_coords': {'left': 3772,
   'right': 3814,
   'top': 2871,
   'bottom': 2924,
   'height': 53,
   'width': 42},
  'begin_index': 1056,
  'id': 'NL-HaNA_1.01.02_3760_0012-page-23-col-1-tr-0-line-52',
  'end_index': 1056},
 {'label': 'textregions',
  'image_coords': {'left': 3532,
   'right': 4478,
   'top': 366,
   'bottom': 2924,
   'height': 2558,
   'width': 946},
  'begin_index': 1004,
  'id': 'NL-HaNA_1.01.02_3760_0012-page-23-col-1-tr-0',
  'end_index': 1056},
 {

In [39]:
annotations_at_index(800)

[{'label': 'lines',
  'image_coords': {'left': 3491,
   'right': 4369,
   'top': 2182,
   'bottom': 2251,
   'height': 69,
   'width': 878},
  'begin_index': 800,
  'id': 'NL-HaNA_1.01.02_3760_0011-page-21-col-1-tr-0-line-38',
  'end_index': 800},
 {'label': 'textregions',
  'image_coords': {'left': 3466,
   'right': 4403,
   'top': 347,
   'bottom': 3014,
   'height': 2667,
   'width': 937},
  'begin_index': 762,
  'id': 'NL-HaNA_1.01.02_3760_0011-page-21-col-1-tr-0',
  'end_index': 816},
 {'label': 'columns',
  'image_coords': {'left': 3462,
   'right': 4403,
   'top': 347,
   'bottom': 3259,
   'height': 2912,
   'width': 941},
  'begin_index': 762,
  'id': 'NL-HaNA_1.01.02_3760_0011-page-21-col-1',
  'end_index': 822},
 {'label': 'sessions',
  'image_coords': None,
  'begin_index': 0,
  'id': 'meeting-1705-01-02-session-1',
  'end_index': 1056},
 {'label': 'scanpage',
  'iiif_url': 'https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0011.jpg/full/,3108/0/def